https://blog.jupyter.org/interactive-gis-in-jupyter-with-ipyleaflet-52f9657fa7a

In [225]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON, LayersControl,
                        Choropleth, Heatmap, SearchControl,FullScreenControl, LegendControl)
from ipywidgets import Text, HTML, widgets, interact, Label, Layout, HBox, VBox
from branca.colormap import linear
import geopandas as gpd
import pandas as pd
import json

from datenguidepy import get_regions, get_statistics, Query

# data from datenguide API

Hesse has region code '06'

In [17]:
get_regions().query("level == 'nuts1'")

,name,level,parent
region_id,,,
10,Saarland,nuts1,DG
11,Berlin,nuts1,DG
12,Brandenburg,nuts1,DG
13,Mecklenburg-Vorpommern,nuts1,DG
14,Sachsen,nuts1,DG
15,Sachsen-Anhalt,nuts1,DG
16,Thüringen,nuts1,DG
01,Schleswig-Holstein,nuts1,DG
02,Hamburg,nuts1,DG


In [18]:
get_statistics()

,short_description,long_description
statistic,,
RLE001,Regelleistungsempfänger,Regelleistungsempfänger\n \n \nErläuterung für...
BEV002,Gestorbene,Gestorbene\n \n \nErläuterung für folgende Sta...
FLCX05,Bodenfläche,Bodenfläche\n \n \nErläuterung für folgende St...
FLC005,Bodenfläche,Bodenfläche\n\n\nErläuterung für folgende Stat...
KIND35,Kinder,Missing
...,...,...
UMS011,Gesamtumsatz des Vorjahres,Gesamtumsatz des Vorjahres\n \n \nErläuterung ...
UMS041,Baugewerblicher Umsatz des Vorjahres,Baugewerblicher Umsatz des Vorjahres\n \n \nEr...
FLC006,Gebietsfläche,Erläuterung für folgende Statistik(en):\nFläch...


In [20]:
selected_stats = 'AI1903'
year = 2013

In [82]:
def get_data_to_show(selected_stats=selected_stats, year = year):
    q = Query.all_regions(parent='06')
    stat = q.add_field(selected_stats)
    stat.add_args({'year': [year]})
    description = stat.description()

    data = q.results(verbose_enums=True, add_units = True)
    data, unit = data[['name', selected_stats]], data[selected_stats+'_unit'].iloc[0]
    # remove Kreis & kreisfreie Stadt from names to be compatible with geojson ids
    data['name'] = data['name'].apply(lambda x: x.split(',')[0])

    data = dict(data.to_dict('split')['data'])
    return data, unit

def get_statistics_description(selected_stats=selected_stats):
    q = Query.all_regions(parent='06')
    stat = q.add_field(selected_stats)
    return stat.description()

# get geoJSON

In [22]:
geojson_data = json.load(open('../data/landkreise_simplify200.geojson','r'))

In [172]:
for feature in geojson_data['features']:
    feature['properties']['name'] = feature['properties']['BEZ']    
#    pop[feature['properties']['GEN']] = feature['properties']['destatis']['population']    

{}

In [24]:
geo_data = {'features':[], 'id':[]}
for f in geojson_data['features']:
    f.update(id=f['properties']['GEN'])
    geo_data['features'].append(f)
#geo_data

# plot data on map

In [78]:
stat_ids = ['AI1903', 'AI1904']
stat_descriptions = [get_statistics_description(si) for si in stat_ids]
stat_tuple = tuple(zip(stat_descriptions, stat_ids))
stat_dict = dict((y, x) for x, y in stat_tuple)

In [215]:
stats_widget = widgets.Dropdown(
    options=stat_tuple,
    value= stat_ids[0],
    description='statistics:',
    disabled=False,
)

year_widget = widgets.Dropdown(
    options= range(2010,2019),
    value=2018,
    description='year:',
    disabled=False,
)

In [261]:
cm = linear.Blues_03

def show_map(selected_stats, year):
    label = Label(layout=Layout(width='100%'))
    html = HTML('''{}'''.format(stat_dict[selected_stats]))
    html.layout.margin = '5px 0px 0px 0px'
    
    control = WidgetControl(widget=html, position='topleft', max_width=200)
    
    choro_data, unit = get_data_to_show(selected_stats, year)
    for f in geo_data['features']:
        f['value'] = choro_data[f['properties']['GEN']]


    layer = Choropleth(
        geo_data= geo_data,
        choro_data= choro_data,
    #     key_on = 'features',
        colormap= cm,
        style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
   )

    geo_json = GeoJSON(data=geo_data,
                       style={'opacity': 0, 'dashArray': '9', 'fillOpacity': 0, 'weight': 1},
                       hover_style={'color': 'green', 'dashArray': '0', 'fillOpacity': 0.7}
                      )
    
    def hover_handler(event=None, feature=None, id=None, properties=None):
        label.value = properties['GEN']       
    geo_json.on_hover(hover_handler)
    
    def update_html(feature,  **kwargs):
        html.value = '''
            {}<br><b>{}</b>: {} {}
        '''.format(stat_dict[selected_stats],
                   feature['properties']['GEN'],
                   feature['value'],
                   unit)
    geo_json.on_hover(update_html)
    
    m = Map(center=(50.5,9), zoom=8)
    m.add_layer(layer)
    m.add_layer(geo_json)
    m.add_control(control)

    m.layout.width = '40%'
    m.layout.height = '700px'

     # custom made "color bar"
    min_value, max_value = min(choro_data.values()), max(choro_data.values())
    legend = LegendControl(
          {str(min_value)+' '+str(unit): cm(0),
          str(min_value+0.5*(max_value-min_value))+' '+str(unit): cm(.5),
          str(max_value)+' '+str(unit): cm(1)},
          name= f"{stat_dict[selected_stats]} ({year})", position="bottomright")
    m.add_control(legend)
    return m#HBox([m, label])#m

In [262]:
interact(show_map, selected_stats=stats_widget, year=year_widget);

interactive(children=(Dropdown(description='statistics:', index=1, options=(('Getrennt erfasste Wertstoffe', '…

In [ ]:
geojson_data

In [ ]:
geo_data